In [1]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from convert_to_tfrecords import *
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.2.0


In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [11]:
training_filenames = "/home/z3u5/Desktop/tfrecords/patches0.tfrecord"
BATCH_SIZE = 32

In [12]:
def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord , num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [13]:
def get_training_dataset(filenames):
    dataset = load_dataset(filenames, labeled=True)
    dataset = dataset.map(prepare_dataset)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [14]:
def prepare_dataset(img1,img2,img3,height,width):
    fraction = 0.8533333
    img1 = tf.image.central_crop(img1, central_fraction = fraction)
    img3 = tf.image.central_crop(img3, central_fraction = fraction)
    return tf.concat([img1,img3],axis = -1),img2
    
    

In [15]:
print("Training data shapes:")
for img1, img2 in get_training_dataset(training_filenames).take(3):
    print(img1.numpy().shape, img2.numpy().shape)

Training data shapes:
(32, 128, 128, 6) (32, 150, 150, 3)
(32, 128, 128, 6) (32, 150, 150, 3)
(32, 128, 128, 6) (32, 150, 150, 3)


In [17]:
num_training_examples = 2500
steps_per_epochs = num_training_examples//BATCH_SIZE

In [18]:
def conv_module(x,filters,conv_filter_size,stride,padding='same'):
    x = tf.keras.layers.Conv2D(filters,conv_filter_size,strides = stride,padding = padding,activation = 'relu')(x)
    x = tf.keras.layers.Conv2D(filters,conv_filter_size,strides = stride,padding = padding,activation = 'relu')(x)
    x = tf.keras.layers.Conv2D(filters,conv_filter_size,strides = stride,padding = padding,activation = 'relu')(x)
    return x

In [19]:
def upsample_module(x,filters,conv_filter_size,stride,upsample_size = (2,2),padding='same'):
    x = tf.keras.layers.UpSampling2D(size = upsample_size,interpolation = 'bilinear')(x)
    x = tf.keras.layers.Conv2D(filters,conv_filter_size,strides = stride,padding = padding,activation = 'relu')(x)
    return x

In [20]:
def generating_kernel(x,kernel_dimension , conv_filter_size, stride, padding, upsample_size):
    x = tf.keras.layers.Conv2D(filters = kernel_dimension,kernel_size = conv_filter_size, strides = stride, padding = padding, activation = 'relu')(x)
    x = tf.keras.layers.Conv2D(filters = kernel_dimension,kernel_size = conv_filter_size, strides = stride, padding = padding, activation = 'relu')(x)
    x = tf.keras.layers.Conv2D(filters = kernel_dimension,kernel_size = conv_filter_size, strides = stride, padding = padding, activation = 'relu')(x)
    x = tf.keras.layers.UpSampling2D(size = upsample_size,interpolation = 'bilinear')(x)
    x = tf.keras.layers.Conv2D(filters = kernel_dimension,kernel_size = conv_filter_size, strides = stride, padding = padding, activation = 'relu')(x)
    return x

In [8]:
class Linear(tf.keras.layers.Layer):

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [1]:
# linear_layer = Linear(32)  # At instantiation, we don't know on what inputs this is going to get called
# y = linear_layer(np.zeros(42).reshape((6,7)),|np.zeros(42).reshape((7,6)))

In [ ]:
# print(y)

In [31]:
# class SeparableConvolutionSlow(tf.keras.layers.Layer):
#     def __init__(self):
#         super(SeparableConvolutionSlow, self).__init__()
        

#     def build(self, input_shape):
        
#         self.outputs = tf.Variable(initial_value=tf.zeros([BATCH_SIZE,128,128,3]),trainable=False)
        
    
#     def call(self, im, vertical, horizontal):
#         n_b = im.shape[0]
#         n_channels = im.shape[-1]
#         m = im.shape[1]
#         m_out = m - 51 + 1
        
#         assert im.shape[1] == im.shape[2]
#         assert vertical.shape[0] == horizontal.shape[0] == n_b
#         assert vertical.shape[-1] == horizontal.shape[-1] == 51
#         assert vertical.shape[1] == horizontal.shape[1] == vertical.shape[2] == horizontal.shape[2] == m_out

#         self.outputs =  tf.Variable(lambda : tf.random.truncated_normal([BATCH_SIZE,128,128,3]),trainable = False)
#         return _sep_conv_worker(im, horizontal, vertical,n_b, self.outputs)  
    
    
    
# def local_separable_conv_2d(im, horizontal, vertical, output=None):

#     n_channels = im.shape[-1]
#     m = im.shape[1]
#     m_out = m - 51 + 1
#     output = tf.Variable(lambda : tf.random.truncated_normal([m_out, m_out,n_channels]))
#     for row in range(m_out):
#         for col in range(m_out):
#             sub_patch = im[row:row + 51, col:col + 51,:]
# #             print(sub_patch.shape)
#             local_horiz = tf.reshape(horizontal[row, col,:],[1,-1])
# #             print(local_horiz.shape)
#             local_vert = tf.reshape(vertical[row, col,:],[-1,1])
# #             print(local_vert.shape)
#             kernel = tf.math.multiply(local_horiz,local_vert)
#             print(tf.reduce_sum(tf.reduce_sum((sub_patch * tf.expand_dims(kernel,-1)),axis = 0),axis = 0).shape)
#             output[row, col,:].assign(tf.reduce_sum(tf.reduce_sum((sub_patch * tf.expand_dims(kernel,-1)),axis = 0),axis = 0))
#             del local_horiz
#             del local_vert
#             del kernel
#             del sub_patch
            
#     return output



# def _sep_conv_worker(im, horizontal, vertical,batch_size, outputs):
#     n_b = batch_size
#     if n_b == None:
#         n_b = 0
#     for b in range(int(n_b)):
#         local_separable_conv_2d(im[b], horizontal[b], vertical[b], output=outputs[b])
        
#     return outputs
    
    
    

    
    



In [10]:
# class SeparableConvolutionSlow(tf.keras.layers.Layer):
#     def __init__(self):
#         super(SeparableConvolutionSlow, self).__init__()
        

#     def build(self, input_shape):
        
#         self.outputs = tf.Variable(initial_value=tf.zeros([BATCH_SIZE,128,128,3]),trainable=False)
        
    
#     def call(self, im, vertical, horizontal):
#         n_b = im.shape[0]
#         n_channels = im.shape[-1]
#         m = im.shape[1]
#         m_out = m - 51 + 1
        
#         assert im.shape[1] == im.shape[2]
#         assert vertical.shape[0] == horizontal.shape[0] == n_b
#         assert vertical.shape[-1] == horizontal.shape[-1] == 51
#         assert vertical.shape[1] == horizontal.shape[1] == vertical.shape[2] == horizontal.shape[2] == m_out

#         return local_separable_conv_2d(im, horizontal, vertical)  
    


    
# def local_separable_conv_2d(im, horizontal, vertical):
#     output_list = []
#     n_channels = im.shape[-1]
#     m = im.shape[1]
#     m_out = m - 51 + 1
#     for row in range(m_out):
#         for col in range(m_out):
#             sub_patch = im[:,row:row + 51, col:col + 51,:]
# #             print(sub_patch.shape)
#             local_horiz = tf.expand_dims(horizontal[:,row, col,:],-2)
# #             print(local_horiz.shape)
#             local_vert = tf.expand_dims(vertical[:,row, col,:],-1)
# #             print(local_vert.shape)
# #             kernel = tf.math.multiply(local_horiz,local_vert)
#             print(tf.reduce_sum(tf.reduce_sum((sub_patch * tf.expand_dims(local_horiz*local_vert,-1)),axis = 1),axis = 1).shape)
#             output_list.append(tf.reduce_sum(tf.reduce_sum((sub_patch * tf.expand_dims(local_horiz*local_vert,-1)),axis = 1),axis = 1))

            
#     return tf.reshape(tf.stack(output_list,axis = 1),(-1,128,128,3))




    
    

    
    



In [38]:
class SeparableConvolutionSlow(tf.keras.layers.Layer):
    def __init__(self):
        super(SeparableConvolutionSlow, self).__init__()
        

    def build(self, input_shape):
        
        self.outputs = tf.Variable(initial_value=tf.zeros([BATCH_SIZE,128,128,3]),trainable=False)
        
    
    def call(self, im, vertical, horizontal):
        n_b = im.shape[0]
        n_channels = im.shape[-1]
        m = im.shape[1]
        m_out = m - 51 + 1
        
        assert im.shape[1] == im.shape[2]
        assert vertical.shape[0] == horizontal.shape[0] == n_b
        assert vertical.shape[-1] == horizontal.shape[-1] == 51
        assert vertical.shape[1] == horizontal.shape[1] == vertical.shape[2] == horizontal.shape[2] == m_out

        return local_separable_conv_2d(im, horizontal, vertical)  
    


    
def local_separable_conv_2d(im, horizontal, vertical):
    output_list = []
    n_channels = im.shape[-1]
    m = im.shape[1]
    m_out = m - 51 + 1
    image_patches = tf.reshape(tf.image.extract_patches(im,sizes = [1,51,51,3],strides = [1,1,1,1],rates = [1,1,1,1],padding = 'VALID'),(-1,128,128,51,51,3))
    output_kernels = tf.expand_dims(tf.math.multiply(tf.expand_dims(horizontal,-2),tf.expand_dims(vertical,-1)),-1)
    output_images = tf.reduce_sum(tf.reduce_sum(image_patches*output_kernels,axis = -2),axis = -2)
    
    
    
    
    
    
    
#     for row in range(m_out):
#         for col in range(m_out):
#             sub_patch = im[:,row:row + 51, col:col + 51,:]
# #             print(sub_patch.shape)
#             local_horiz = tf.expand_dims(horizontal[:,row, col,:],-2)
# #             print(local_horiz.shape)
#             local_vert = tf.expand_dims(vertical[:,row, col,:],-1)
# #             print(local_vert.shape)
# #             kernel = tf.math.multiply(local_horiz,local_vert)
#             print(tf.reduce_sum(tf.reduce_sum((sub_patch * tf.expand_dims(local_horiz*local_vert,-1)),axis = 1),axis = 1).shape)
#             output_list.append(tf.reduce_sum(tf.reduce_sum((sub_patch * tf.expand_dims(local_horiz*local_vert,-1)),axis = 1),axis = 1))

            
    return output_images




    
    

    
    



In [39]:
def custom_loss(y_true, y_pred):
 
    
    return tf.norm(tf.norm(y_true-y_pred, ord=1, axis=(1,2)),axis = 1)

In [40]:
def create_model(input_shape = (128,128,6)):
    conv_filter_size = (3, 3)
    stride = (1, 1)
    padding = 'same'
    upsample_size = (2,2)
    kernel_dimension = 51
    x_input = tf.keras.Input(input_shape)
    x = x_input
    pad_dimension = kernel_dimension//2
    i1 = x[:,:,:,0:3]
    i_out = i1
    i2 = x[:,:,:,3:6]
    i1 = tf.pad(i1,[[0,0],[pad_dimension,pad_dimension],[pad_dimension,pad_dimension],[0,0]])
    i2 = tf.pad(i2,[[0,0],[pad_dimension,pad_dimension],[pad_dimension,pad_dimension],[0,0]])
    AvgPooling = tf.keras.layers.AveragePooling2D()
    
    x = conv_module(x,32,conv_filter_size,stride,padding)
    x = AvgPooling(x)
    
    
    x_64 = conv_module(x,64,conv_filter_size,stride,padding)
    x_128 = AvgPooling(x_64)
    
    
    x_128 = conv_module(x_128,128,conv_filter_size,stride,padding)
    x_256 = AvgPooling(x_128)
    
    x_256 = conv_module(x_256,256,conv_filter_size,stride,padding)
    x_512 = AvgPooling(x_256)
    
    x_512 = conv_module(x_512,512,conv_filter_size,stride,padding)
    x = AvgPooling(x_512)
    
    x = conv_module(x,512,conv_filter_size,stride,padding)
    
    
    
    x = upsample_module(x,512,conv_filter_size,stride,upsample_size,padding)
    x += x_512
    x = conv_module(x,256,conv_filter_size,stride,padding)
    
    x = upsample_module(x,256,conv_filter_size,stride,upsample_size,padding)
    x += x_256
    x = conv_module(x,128,conv_filter_size,stride,padding)

    x = upsample_module(x,128,conv_filter_size,stride,upsample_size,padding)
    x += x_128
    x = conv_module(x,64,conv_filter_size,stride,padding)
    
    x = upsample_module(x,64,conv_filter_size,stride,upsample_size,padding)
    x += x_64
#     print(x)
    
    k1h = generating_kernel(x,kernel_dimension , conv_filter_size, stride, padding, upsample_size)
    k1v = generating_kernel(x,kernel_dimension , conv_filter_size, stride, padding, upsample_size)
    k2h = generating_kernel(x,kernel_dimension , conv_filter_size, stride, padding, upsample_size)
    k2v = generating_kernel(x,kernel_dimension , conv_filter_size, stride, padding, upsample_size)
    
    
    image_patches = tf.reshape(tf.image.extract_patches(i1,sizes = [1,51,51,3],strides = [1,1,1,1],rates = [1,1,1,1],padding = 'VALID'),(-1,128,128,51,51,3))
#     output_kernels = tf.expand_dims(tf.math.multiply(tf.expand_dims(horizontal,-2),tf.expand_dims(vertical,-1)),-1)
    output_images = tf.reduce_sum(tf.reduce_sum(image_patches*tf.expand_dims(horizontal,-2)*tf.expand_dims(vertical,-1)),-1),axis = -2),axis = -2)
    
    
    return tf.keras.Model(inputs = x_input, outputs = i_out, name = 'IIASC')
    
#     x = self.conv32(x)
#     x = self.pool(x)

#     x64 = self.conv64(x)
#     x128 = self.pool(x64)

#     x128 = self.conv128(x128)
#     x256 = self.pool(x128)

#     x256 = self.conv256(x256)
#     x512 = self.pool(x256)

#     x512 = self.conv512(x512)
#     x = self.pool(x512)

#     x = self.conv512x512(x)

#     # ------------ Expansion ------------

#     x = self.upsamp512(x)
#     x += x512
#     x = self.upconv256(x)

#     x = self.upsamp256(x)
#     x += x256
#     x = self.upconv128(x)

#     x = self.upsamp128(x)
#     x += x128
#     x = self.upconv64(x)

#     x = self.upsamp64(x)
#     x += x64

#     # ------------ Final branches ------------

#     k2h = self.upconv51_1(x)

#     k2v = self.upconv51_2(x)

#     k1h = self.upconv51_3(x)

#     k1v = self.upconv51_4(x)

#     padded_i2 = self.pad(i2)
#     padded_i1 = self.pad(i1)
#     self.pool = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))

In [41]:
model = create_model()
model.summary()

Model: "IIASC"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 6) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 128, 128, 32) 1760        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 128, 128, 32) 9248        conv2d_188[0][0]                 
__________________________________________________________________________________________________
conv2d_190 (Conv2D)             (None, 128, 128, 32) 9248        conv2d_189[0][0]                 
______________________________________________________________________________________________

In [43]:
# with strategy.scope():

    
#     model = tf.keras.Sequential([
#         tf.keras.layers.Conv2D(),
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])
        
model.compile(
    optimizer='adamax',
    loss = custom_loss,
    metrics=['accuracy']
)
model.summary()

Model: "IIASC"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 6) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 128, 128, 32) 1760        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 128, 128, 32) 9248        conv2d_188[0][0]                 
__________________________________________________________________________________________________
conv2d_190 (Conv2D)             (None, 128, 128, 32) 9248        conv2d_189[0][0]                 
______________________________________________________________________________________________

In [44]:
history = model.fit(get_training_dataset(training_filenames), steps_per_epoch=steps_per_epochs, epochs=20)

Epoch 1/20


UnimplementedError:  Only support ksizes across space.
	 [[node IIASC/separable_convolution_slow_4/ExtractImagePatches (defined at <ipython-input-38-d64379cf053b>:32) ]] [Op:__inference_train_function_14709]

Errors may have originated from an input operation.
Input Source operations connected to node IIASC/separable_convolution_slow_4/ExtractImagePatches:
 IIASC/tf_op_layer_Pad_8/Pad_8 (defined at <ipython-input-44-60b7f27fec54>:1)

Function call stack:
train_function


In [33]:
tf.extract_image_patches()

AttributeError: module 'tensorflow' has no attribute 'extract_image_patches'